In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import glob
import datetime as dt
%matplotlib inline

In [2]:
files = glob.glob("*.csv")
print(files)

#but some datasets have different timeframes

['particulate_matter.csv', 'ozone.csv', 'black_carbon.csv', 'nitric_oxide.csv', 'sulfur_dioxide.csv', 'nitrogen_dioxide.csv', 'black_carbon_mean.csv']


black carbon - summer 2012 to summer 2018
nitric oxide - summer 2013 to summer 2019
nitrogen dioxide - summer 2013 to summer 2019
particulate matter - summer 2013 to summer 2019
sulfur dioxide - winter 2008 to winter 2014
ozone - summer 2009 to summer 2019

In [3]:
#reframe the data to extract only mean values for each time interval per community district

cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

df = pd.read_csv('nitric_oxide.csv')
#extract only CD data
df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                     df.columns[4]:'CD',
                                     df.columns[8]:'pol_mean'})

#split 'Summer 2018' into 'Summer' and '2018'
new = df['Year'].str.split(" ", n=1, expand=True) 
df['Year'] = new[1]
df['Month'] = new[0]
df['Day'] = 1

#convert Summer rows into month==5, Winter rows into month==11
for i in range(len(df)):
    if df.iloc[i,6] == 'Summer':
        df.iloc[i,6] = 5
    else:
        df.iloc[i,6] = 11
    
    df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1]) #regex to remove '-'
    
    if '-' in df.iloc[i,0]:
        df.iloc[i,0] = df.iloc[i,0][:4]
    
df2 = df[['Year','Month','Day']]
df['Date'] = pd.to_datetime(df2).dt.date

df = df.iloc[:,[8,1,2,5]]
df = df.pivot_table(values='pol_mean', index=['CD','CDname'], columns='Date')
bc = df.reset_index()

bc.head()

Date,CD,CDname,2013-05-01,2013-11-01,2014-05-01,2014-11-01,2015-05-01,2015-11-01,2016-05-01,2016-11-01,2017-05-01,2017-11-01,2018-05-01,2018-11-01,2019-05-01
0,101,Financial District,20.1,49.5,23.9,38.6,19.7,35.1,15.5,28.2,13.3,31.1,12.1,21.9,10.1
1,102,Greenwich Village and Soho,21.9,58.4,30.3,44.1,21.6,43.6,22.2,36.3,19.3,38.3,17.6,23.7,10.6
2,103,Lower East Side and Chinatown,16.5,49.9,22.8,37.3,16.5,36.5,17.2,29.4,15.0,29.9,13.0,21.0,8.8
3,104,Clinton and Chelsea,20.9,52.0,26.1,39.2,20.0,37.2,16.6,30.6,15.0,30.9,13.8,22.9,10.1
4,105,Midtown,30.5,63.0,36.2,49.3,28.7,41.1,18.7,33.5,16.9,33.2,14.8,25.5,11.0


In [4]:
bc.to_csv('nitric_oxide.csv')

In [22]:
cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

df = pd.read_csv('black_carbon.csv')
df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

pol = df.iloc[0,5]
df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                     df.columns[4]:'CD',
                                     df.columns[6]:'{}-perc10'.format(pols[pol]),
                                     df.columns[7]:'{}-perc90'.format(pols[pol]),
                                     df.columns[8]:'{}-mean'.format(pols[pol])})

new = df['Year'].str.split(" ", n=1, expand=True) 
df['Year'] = new[1]
df['Month'] = new[0]
df['Day'] = 1

for i in range(len(df)):
    if df.iloc[i,6] == 'Summer':
        df.iloc[i,6] = 5
    else:
        df.iloc[i,6] = 11
    
    df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1])
    
    if '-' in df.iloc[i,0]:
        df.iloc[i,0] = df.iloc[i,0][:4]
    
df2 = df[['Year','Month','Day']]
df['Date'] = pd.to_datetime(df2)

df = df.iloc[:,[8,1,2,3,4,5]]
maindf = df
maindf.head()

,Date,CDname,CD,BC-perc10,BC-perc90,BC-mean
0,2012-05-01,Bay Ridge and Dyker Heights,310,0.9,1.1,1.0
1,2013-05-01,Bay Ridge and Dyker Heights,310,0.8,1.1,0.9
2,2014-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.9
3,2015-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.9
4,2016-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.8


In [9]:
cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

for file in files:
    
    if file != 'black_carbon.csv':
        df = pd.read_csv(file)
        df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

        pol = df.iloc[0,5]
        df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                             df.columns[4]:'CD',
                                             df.columns[6]:'{}-perc10'.format(pols[pol]),
                                             df.columns[7]:'{}-perc90'.format(pols[pol]),
                                             df.columns[8]:'{}-mean'.format(pols[pol])})

        new = df['Year'].str.split(" ", n=1, expand=True) 
        df['Year'] = new[1]
        df['Month'] = new[0]
        df['Day'] = 1

        for i in range(len(df)):
            if df.iloc[i,6] == 'Summer':
                df.iloc[i,6] = 5
            else:
                df.iloc[i,6] = 11

            df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1])

            if '-' in df.iloc[i,0]:
                df.iloc[i,0] = df.iloc[i,0][:4]

        df2 = df[['Year','Month','Day']]
        df['Date'] = pd.to_datetime(df2)

        df = df.iloc[:,[8,1,2,3,4,5]]
        
        maindf = maindf.merge(df,on=['CDname','CD','Date'],how='outer')
        
    else:
        pass

In [10]:
maindf.head()

,Date,CDname,CD,BC-perc10,BC-perc90,BC-mean,PM2.5-perc10_x,PM2.5-perc90_x,PM2.5-mean_x,PM2.5-perc10_y,...,O3-mean,NO-perc10,NO-perc90,NO-mean,SO2-perc10,SO2-perc90,SO2-mean,NO2-perc10,NO2-perc90,NO2-mean
0,2012-05-01,Bay Ridge and Dyker Heights,310,0.9,1.1,1.0,NaN,NaN,NaN,NaN,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-05-01,Bay Ridge and Dyker Heights,310,0.8,1.1,0.9,9.4,10.6,9.9,9.4,...,30.3,6.1,13.7,9.2,NaN,NaN,NaN,11.0,17.9,15.2
2,2014-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.9,8.1,9.2,8.5,8.1,...,30.7,7.4,15.9,11.0,NaN,NaN,NaN,10.9,16.9,14.5
3,2015-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.9,8.7,9.9,9.2,8.7,...,31.8,6.7,14.6,9.8,NaN,NaN,NaN,10.7,16.5,14.2
4,2016-05-01,Bay Ridge and Dyker Heights,310,0.7,1.0,0.8,7.7,8.9,8.2,7.7,...,33.5,6.3,9.5,7.6,NaN,NaN,NaN,10.7,16.6,14.3


In [11]:
len(maindf.CD.unique())

59

In [12]:
len(maindf.Date.unique())

22

In [13]:
maindf.CD.unique()

array([310, 411, 303, 206, 311, 312, 316, 304, 110, 104, 313, 308, 317,
       111, 305, 404, 101, 314, 318, 407, 205, 302, 301, 102, 204, 408,
       202, 403, 412, 409, 207, 401, 103, 105, 109, 211, 203, 201, 306,
       209, 413, 406, 405, 208, 414, 315, 502, 309, 410, 501, 106, 307,
       210, 503, 108, 107, 112, 212, 402])

In [14]:
#maindf.to_csv('pollutants.csv')